In [1]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-york-city-taxi-fare-prediction/sample_submission.csv
/kaggle/input/new-york-city-taxi-fare-prediction/GCP-Coupons-Instructions.rtf
/kaggle/input/new-york-city-taxi-fare-prediction/train.csv
/kaggle/input/new-york-city-taxi-fare-prediction/test.csv


### Setp 1
- Install req libs and Load the datas

In [2]:
import numpy as np
import pandas as pd

In [3]:
data_dir = '/kaggle/input/new-york-city-taxi-fare-prediction/'

In [4]:
!ls -lh {data_dir}

total 5.4G
-rw-r--r-- 1 nobody nogroup  486 Mar 26 17:41 GCP-Coupons-Instructions.rtf
-rw-r--r-- 1 nobody nogroup 336K Mar 26 17:41 sample_submission.csv
-rw-r--r-- 1 nobody nogroup 960K Mar 26 17:41 test.csv
-rw-r--r-- 1 nobody nogroup 5.4G Mar 26 17:44 train.csv


All the data is in the **train.csv** file

In [5]:
# this will take too much tine
# !wc -l {data_dir}/train.csv

In [6]:
!wc -l {data_dir}/test.csv

9914 /kaggle/input/new-york-city-taxi-fare-prediction//test.csv


Getting the first line of the dataset using the _shell_ command

In [7]:
    !head {data_dir}/train.csv

In [8]:
!head {data_dir}/test.csv

fare amount is missingw which is the pratical Target column for our analysis

#### Loading
Loading the entire dataset into Pandas is going to be slow, so we can use the following optimizations:

- Ignore the key column
- Parse pickup datetime while loading data
- Specify data types for other columns
    - float32 for geo coordinates
    - float32 for fare amount
    - uint8 for passenger count
- Work with a 1% sample of the data (~500k rows)

In [9]:
# Taking all the columns except the keys
selected_cols = 'fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count'.split(',')
selected_cols

['fare_amount',
 'pickup_datetime',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'passenger_count']

In [10]:
d_types = {
    'fare_amount' : 'float32',
    'pickup_longitude': 'float32',
    'pickup_latitude': 'float32',
    'dropoff_longitude': 'float32',
    'dropoff_latitude' : 'float32',
    'passenger_count' : 'uint8',
}
d_types

{'fare_amount': 'float32',
 'pickup_longitude': 'float32',
 'pickup_latitude': 'float32',
 'dropoff_longitude': 'float32',
 'dropoff_latitude': 'float32',
 'passenger_count': 'uint8'}

In [11]:
import random as rnd
sample_fraction = 0.01
rnd.seed(42)

def skip_row(row_idx) :
#     If this functon retrun false then the number is going to be in data
    if row_idx == 0 :
        return False
    return rnd.random() > sample_fraction

In [12]:
data = pd.read_csv(data_dir + '/train.csv', 
                   usecols = selected_cols, 
                   dtype = d_types, 
                   parse_dates= ['pickup_datetime'], 
                   skiprows = skip_row) # A function is passed as a prameter

In [13]:
data.sample(10)

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
159209,8.9,2011-11-01 09:53:00+00:00,-73.959625,40.814026,-73.942818,40.840816,5
321214,3.5,2012-12-30 00:08:13+00:00,-73.955276,40.788662,-73.964348,40.792484,1
436932,6.5,2009-09-29 20:09:42+00:00,-73.975021,40.759159,-73.959724,40.782200,1
151063,4.5,2010-06-09 18:12:07+00:00,-73.987434,40.738144,-73.994606,40.727825,1
189252,12.5,2014-06-15 09:49:00+00:00,-73.985008,40.747250,-74.011131,40.729183,2
480629,3.7,2011-07-05 13:58:00+00:00,-74.006950,40.728928,-74.004936,40.737392,5
485475,36.5,2014-03-27 21:39:35+00:00,-73.981674,40.776100,-73.974510,40.671085,1
177407,7.5,2014-01-27 19:05:00+00:00,-74.001747,40.740959,-73.983536,40.728371,3
80828,12.1,2012-03-02 23:42:00+00:00,-73.990440,40.775394,-73.944473,40.779797,1
396164,4.1,2012-08-13 21:24:42+00:00,-73.992867,40.731792,-74.002357,40.733425,1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552450 entries, 0 to 552449
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   fare_amount        552450 non-null  float32            
 1   pickup_datetime    552450 non-null  datetime64[ns, UTC]
 2   pickup_longitude   552450 non-null  float32            
 3   pickup_latitude    552450 non-null  float32            
 4   dropoff_longitude  552450 non-null  float32            
 5   dropoff_latitude   552450 non-null  float32            
 6   passenger_count    552450 non-null  uint8              
dtypes: datetime64[ns, UTC](1), float32(5), uint8(1)
memory usage: 15.3 MB


In [15]:
data.shape

(552450, 7)

In [16]:
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,552450.000000,552450.000000,552450.000000,552450.000000,552450.000000,552450.000000
mean,11.354059,-72.497063,39.910500,-72.504326,39.934265,1.684983
std,9.811924,11.618246,8.061114,12.074346,9.255057,1.337664
min,-52.000000,-1183.362793,-3084.490234,-3356.729736,-2073.150635,0.000000
25%,6.000000,-73.992020,40.734875,-73.991425,40.733990,1.000000
50%,8.500000,-73.981819,40.752621,-73.980179,40.753101,1.000000
75%,12.500000,-73.967155,40.767036,-73.963737,40.768059,2.000000
max,499.000000,2420.209473,404.983337,2467.752686,3351.403076,208.000000


Here I am trying with 1% if the data, We can also try this using _3%, 10%, 50% and even 100%_ of the data

In [17]:
test_data = pd.read_csv(data_dir + '/test.csv', dtype= d_types, parse_dates = ['pickup_datetime'])
test_data.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-01-27 13:08:24.0000002,2015-01-27 13:08:24+00:00,-73.973320,40.763805,-73.981430,40.743835,1
1,2015-01-27 13:08:24.0000003,2015-01-27 13:08:24+00:00,-73.986862,40.719383,-73.998886,40.739201,1
2,2011-10-08 11:53:44.0000002,2011-10-08 11:53:44+00:00,-73.982521,40.751259,-73.979652,40.746140,1
3,2012-12-01 21:12:12.0000002,2012-12-01 21:12:12+00:00,-73.981163,40.767807,-73.990448,40.751637,1
4,2012-12-01 21:12:12.0000003,2012-12-01 21:12:12+00:00,-73.966049,40.789776,-73.988564,40.744427,1


In [18]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   key                9914 non-null   object             
 1   pickup_datetime    9914 non-null   datetime64[ns, UTC]
 2   pickup_longitude   9914 non-null   float32            
 3   pickup_latitude    9914 non-null   float32            
 4   dropoff_longitude  9914 non-null   float32            
 5   dropoff_latitude   9914 non-null   float32            
 6   passenger_count    9914 non-null   uint8              
dtypes: datetime64[ns, UTC](1), float32(4), object(1), uint8(1)
memory usage: 319.6+ KB
